# RFC Simulation Lab (Modules A–Q)This notebook lets you run **any** RFC module A–Q dynamically:- Select a module- Load its parameters from `SimulationConfigs.json`- Simulate fields / ODEs / series- Plot the results and print numeric outputs

In [ ]:
# Imports
import json5
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
from pathlib import Path
from ipywidgets import interact, Dropdown
from IPython.display import display

# Symbolic setup (for any sympy if needed)
t = sp.symbols('t')

In [ ]:
# Load all configs from the JSON in this same folder
config_path = Path('SimulationConfigs.json')
with open(config_path, 'r') as f:
    all_configs = json5.load(f)

# Build a dropdown of module names
module_options = sorted(cfg['module'] for cfg in all_configs)
module_selector = Dropdown(options=module_options, description='Module:')
display(module_selector)

In [ ]:
def run_simulation(selected_module):
    cfg = next(c for c in all_configs if c['module']==selected_module)
    print(f"=== Module {selected_module}: {cfg.get('description','')} ===\n")

    # Common time grid
    dt = cfg.get('dt', 0.1)
    tmin, tmax = cfg.get('t_range', [0, 60])
    t_vals = np.arange(tmin, tmax, dt)
    delta = cfg.get('delta', 4.669)
    alpha = cfg.get('alpha', 0.01)

    # --- A: Friedmann ---
    if selected_module == 'A':
        lam = cfg.get('lambda', 1.0)
        def psiChi(t):    return 0.002*np.cos(0.1*t)+0.001*np.sin(0.2*t)
        def gammaZeta(t): return 0.002*np.exp(-0.05*t)+0.0005*np.cos(0.03*t)

        a    = np.zeros_like(t_vals)
        adot = np.zeros_like(t_vals)
        a[0]    = cfg['initial_conditions']['a(0)']
        adot[0] = cfg['initial_conditions']["a'(0)"]

        for i in range(1, len(t_vals)):
            ti = t_vals[i-1]
            sum_term = sum(np.exp(-alpha*j*ti)/delta**j for j in range(1,41))
            acc = -lam*(a[i-1]**3) + sum_term + psiChi(ti) + gammaZeta(ti)
            adot[i] = adot[i-1] + acc*dt
            a[i]    = a[i-1]   + adot[i]*dt

        plt.figure(figsize=(8,4))
        plt.plot(t_vals, a,    label='a(t)')
        plt.plot(t_vals, adot, label="a'(t)")
        plt.title('Module A: Recursive Friedmann')
        plt.legend(); plt.xlabel('t'); plt.show()

        print(f"Final a({tmax})   = {a[-1]:.6f}")
        print(f"Final a'({tmax})  = {adot[-1]:.6f}\n")

    # --- B: CP-Phase Soliton ---
    elif selected_module == 'B':
        lam   = cfg['lambda']
        theta = np.zeros_like(t_vals)
        thetad= np.zeros_like(t_vals)
        theta[0]  = cfg['initial_conditions']['Θ(0)']
        thetad[0] = cfg['initial_conditions']["Θ'(0)"]

        for i in range(1, len(t_vals)):
            tdd = -lam*np.sin(theta[i-1])
            thetad[i] = thetad[i-1] + tdd*dt
            theta[i]  = theta[i-1] + thetad[i]*dt

        plt.figure(figsize=(8,4))
        plt.plot(t_vals, theta)
        plt.title('Module B: CP-Phase Soliton')
        plt.xlabel('t'); plt.ylabel('Θ(t)'); plt.show()

        print(f"Final Θ({tmax}) = {theta[-1]:.6f}\n")

    # --- C: Ringdown Echo ---
    elif selected_module == 'C':
        nu = cfg.get('nu', 0.005)
        psi = np.array([
            sum((1/delta**j)*np.cos(j*t + nu)*np.exp(-alpha*j*t) for j in range(1,41))
            for t in t_vals
        ])

        plt.figure(figsize=(8,4))
        plt.plot(t_vals, psi)
        plt.title('Module C: Gravitational Ringdown Echo')
        plt.xlabel('t'); plt.ylabel('ψ(t)'); plt.show()

        print(f"ψ(t) min = {psi.min():.6f}, max = {psi.max():.6f}\n")

    # --- D: Entropy Collapse + Identity ---
    elif selected_module == 'D':
        S   = np.array([
            -sum((1/delta**j)*np.log(1/delta**j)*np.exp(-alpha*j*t)
                 for j in range(1,41))
            for t in t_vals
        ])
        psi = np.zeros_like(t_vals)
        pdot= np.zeros_like(t_vals)
        psi[0] = cfg['initial_conditions']['ψ(0)']
        beta   = cfg.get('beta', 0.8)
        def pChi(t): return 0.01*np.cos(0.2*t)
        def oNu(t): return 0.01*np.exp(-0.05*t)

        for i in range(1, len(t_vals)):
            ti     = t_vals[i-1]
            pdd    = -4*psi[i-1]**3 + beta*psi[i-1] - pChi(ti) - oNu(ti)
            pdot[i]= pdot[i-1] + pdd*dt
            psi[i] = psi[i-1] + pdot[i]*dt

        fig, ax = plt.subplots(1,2,figsize=(12,3))
        ax[0].plot(t_vals, S);   ax[0].set_title('S(t)')
        ax[1].plot(t_vals, psi); ax[1].set_title('ψ_self(t)')
        plt.show()

        print(f"Final S = {S[-1]:.6f}")
        print(f"Final ψ_self = {psi[-1]:.6f}\n")
    # --- E: Neural Fractal PDE ---
    elif selected_module == 'E':
        lam = cfg['lambda']
        phi = np.zeros_like(t_vals)
        pd  = np.zeros_like(t_vals)
        for i in range(1, len(t_vals)):
            phi_dd = -lam*phi[i-1] + 0.01
            pd[i]  = pd[i-1] + phi_dd*dt
            phi[i] = phi[i-1] + pd[i]*dt

        plt.figure(figsize=(8,4))
        plt.plot(t_vals, phi)
        plt.title('Module E: Neural Fractal PDE')
        plt.xlabel('t'); plt.ylabel('ϕ(t)'); plt.show()

        print(f"Final ϕ = {phi[-1]:.6f}\n")
    # --- F: Observer Identity ---
    elif selected_module == 'F':
        psi = np.zeros_like(t_vals)
        pd  = np.zeros_like(t_vals)
        psi[0] = cfg['initial_conditions']['ψ(0)']
        beta   = cfg['beta']
        def pChi(t): return 0.1*np.cos(0.2*t)
        def oNu(t): return 0.01*np.exp(-0.05*t)

        for i in range(1, len(t_vals)):
            pdd   = -4*psi[i-1]**3 + beta*psi[i-1] - pChi(t_vals[i-1]) - oNu(t_vals[i-1])
            pd[i] = pd[i-1] + pdd*dt
            psi[i]= psi[i-1] + pd[i]*dt

        plt.figure(figsize=(8,4))
        plt.plot(t_vals, psi)
        plt.title('Module F: Observer Identity')
        plt.show()

        print(f"Final ψ_self = {psi[-1]:.6f}\n")

    # --- G: MCMC Observables ---
    elif selected_module == 'G':
        eps_min, eps_max = cfg['prior_ranges']['epsilon']
        eps_vals = np.linspace(eps_min, eps_max, 200)
        Yp   = 0.246 + 0.015*np.exp(-1000*eps_vals)
        Df   = 2.4   + 0.05*np.sin(500*eps_vals)
        Echo = 18    + 2*np.cos(200*eps_vals)

        plt.figure(figsize=(8,4))
        plt.plot(eps_vals, Yp,   label='Y_p')
        plt.plot(eps_vals, Df,   label='D_f')
        plt.plot(eps_vals, Echo, label='EchoDelay')
        plt.title('Module G: RFC Observables')
        plt.legend(); plt.show()
        bf = cfg.get('best_fit')
        if bf:
            print("Best-fit parameters:")
            for k,v in bf.items():
                print(f"  {k} = {v}")
            print()

    # --- H: Spin Foam Partition ---
    elif selected_module == 'H':
        jmax = 40
        Sigma = np.array([1 + 0.2*np.sin(np.pi*j) for j in range(1,jmax+1)])
        Av    = np.array([(1/delta)**j * np.exp(-alpha*j) for j in range(1,jmax+1)])
        Z     = Sigma * Av * (2*np.arange(1,jmax+1)+1)

        plt.figure(figsize=(8,4))
        plt.plot(np.arange(1,jmax+1), Z)
        plt.title('Module H: Spin Foam Partition')
        plt.xlabel('j'); plt.ylabel('Z_j'); plt.show()

        print(f"Total Partition Sum = {Z.sum():.6e}\n")

    # --- I: Symbolic Mass Field ---
    elif selected_module == 'I':
        for nu in cfg['nu_values']:
            psi = np.array([
                sum((1/delta**j)*np.cos(j*t + nu)*np.exp(-alpha*j*t) for j in range(1,41))
                for t in t_vals
            ])
            m_rms = np.sqrt(np.mean(psi**2))
            plt.plot(t_vals, psi, label=f'ν={nu}')
            print(f"ν={nu} → m_rms = {m_rms:.6e}")
        plt.title('Module I: Mass Field'); plt.legend(); plt.show()
        print()

    # --- J: Decoherence Divergence ---
    elif selected_module == 'J':
        nu1, nu2 = cfg['nu_values']
        psi1 = np.array([sum((1/delta**j)*np.cos(j*t+nu1)*np.exp(-alpha*j*t) for j in range(1,41)) for t in t_vals])
        psi2 = np.array([sum((1/delta**j)*np.cos(j*t+nu2)*np.exp(-alpha*j*t) for j in range(1,41)) for t in t_vals])
        lam_div = np.log(1 + np.abs(psi1 - psi2))

        plt.figure(figsize=(8,4))
        plt.plot(t_vals, lam_div)
        plt.title('Module J: Decoherence Divergence')
        plt.show()

        print(f"Max λ_div = {lam_div.max():.6f}\n")

    # --- K: Collapse–Rebirth Field ---
    elif selected_module == 'K':
        nu = cfg.get('nu', 0.003)
        psi_reb = np.array([
            sum((1/delta**j)*np.sin(j*t + j**2)*np.exp(-alpha*j*t) for j in range(1,41))
            for t in t_vals
        ])

        plt.figure(figsize=(8,4))
        plt.plot(t_vals, psi_reb)
        plt.title('Module K: Collapse–Rebirth Field')
        plt.show()

        print(f"Final ψ_rebirth = {psi_reb[-1]:.6f}\n")

    # --- L: RFC vs Hidden-Variable Entropy ---
    elif selected_module == 'L':
        nu   = cfg['nu_values'][0]
        lamv = cfg['lambda_values'][0]
        psi_rfc = np.array([sum((1/delta**j)*np.cos(j*t+nu)*np.exp(-alpha*j*t) for j in range(1,41)) for t in t_vals])
        psi_hv  = np.cos(lamv*t_vals + lamv)
        Sr = -(psi_rfc**2)*np.log(np.abs(psi_rfc**2)+1e-10)
        Sh = -(psi_hv**2)*np.log(np.abs(psi_hv**2)+1e-10)

        plt.figure(figsize=(8,4))
        plt.plot(t_vals, Sr, label='RFC S')
        plt.plot(t_vals, Sh, label='HV S')
        plt.title('Module L: Entropy vs Hidden Var')
        plt.legend(); plt.show()

        print(f"Max S_RFC = {Sr.max():.6f}, Max S_HV = {Sh.max():.6f}\n")

    # --- M: Mass & Energy Emergence ---
    elif selected_module == 'M':
        nu   = cfg['nu']
        psi  = np.array([sum((1/delta**j)*np.cos(j*t+nu)*np.exp(-alpha*j*t) for j in range(1,41)) for t in t_vals])
        psid = np.gradient(psi, dt)
        E    = psid**2 + psi**2
        m_rms= np.sqrt(np.mean(psi**2))

        print(f"RMS mass = {m_rms:.6e}")
        plt.figure(figsize=(8,4))
        plt.plot(t_vals, E)
        plt.title('Module M: Energy Curve E(t)')
        plt.show()
        print()

    # --- N: Symbolic Constants Emergence ---
    elif selected_module == 'N':
        for nu in cfg['nu_values']:
            psi  = np.array([sum((1/delta**j)*np.cos(j*t+nu)*np.exp(-alpha*j*t) for j in range(1,41)) for t in t_vals])
            psid = np.gradient(psi, dt)
            E    = psid**2 + psi**2
            m_rms= np.sqrt(np.mean(psi**2))
            αG   = m_rms**2
            αEM  = 1/np.mean(E)
            Λs   = 1/np.sum(E)
            print(f"ν={nu} → α_G={αG:.3e}, α_EM={αEM:.3e}, Λ_symbolic={Λs:.3e}")
        print()

    # --- O: Baryogenesis from Curvature ---
    elif selected_module == 'O':
        nu   = cfg['nu']
        psi  = np.array([sum((1/delta**j)*np.cos(j*t+nu)*np.exp(-alpha*j*t) for j in range(1,41)) for t in t_vals])
        kappa = np.gradient(np.gradient(psi, dt), dt)
        a_rec = np.array([sum(np.exp(-alpha*j*t)/delta**j for j in range(1,41)) for t in t_vals])
        T_rec = np.gradient(np.log(a_rec), dt)
        etaBs= np.gradient(kappa, dt)/(T_rec**3 + 1e-10)

        plt.figure(figsize=(8,4))
        plt.plot(t_vals, etaBs)
        plt.title('Module O: Baryon Asymmetry η_B/S')
        plt.show()

        print(f"Mean η_B/S = {etaBs.mean():.6e}, Max = {etaBs.max():.6e}\n")

    # --- P: B vs D Meson Entropy Spike ---
    elif selected_module == 'P':
        for nu in cfg['nu_values']:
            psiB = np.array([sum((1/delta**j)*np.cos(j*t+nu)*np.exp(-alpha*j*t) for j in range(1,41)) for t in t_vals])
            psiD = np.array([sum((1/delta**j)*np.cos(j*t-nu)*np.exp(-alpha*j*t) for j in range(1,41)) for t in t_vals])
            mm   = psiB - psiD
            Ssp  = -(mm**2)*np.log(np.abs(mm**2)+1e-10)
            plt.plot(t_vals, Ssp, label=f'ν={nu}')
            print(f"ν={nu} → Spike max = {Ssp.max():.6f}")
        plt.title('Module P: Entropy Spike'); plt.legend(); plt.show()
        print()

    # --- Q: Scalar-Induced Decoherence ---
    elif selected_module == 'Q':
        msc  = cfg['parameters']['m_scalar']
        nu   = cfg['nu']
        psi_s= np.array([sum((1/delta**j)*np.cos(j*t+nu)*np.exp(-alpha*j*t)*np.exp(-msc*t) for j in range(1,41)) for t in t_vals])
        psi_o= np.array([sum((1/delta**j)*np.sin(j*t+nu**2)*np.exp(-alpha*j*t) for j in range(1,41)) for t in t_vals])
        lamd = np.log(1 + np.abs(psi_s - psi_o))

        plt.figure(figsize=(8,4))
        plt.plot(t_vals, lamd)
        plt.title('Module Q: Scalar-Induced Decoherence')
        plt.show()

        print(f"Max λ_div = {lamd.max():.6f}\n")

    else:
        print("Unknown module! Check your JSON!")

# Launch the interactive selector
interact(run_simulation, selected_module=module_selector)